In [110]:
import numpy as np
from scipy.special import logsumexp

In [5]:
path_to_data = '../datasets/mnist.npz'

In [94]:
def load_train_test_data(path):
    with np.load(path) as f:
        x_train = f['x_train']
        x_test = f['x_test']
        y_train = f['y_train']
        y_test = f['y_test']
    return x_train, x_test, y_train, y_test

In [95]:
x_train, x_test, y_train, y_test = load_train_test_data(path_to_data)

In [102]:
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
class GaussianNB:
    def __init__(self):
        self.means = None
        self.stds = None
        self.priors = None
    
    def fit(self, X, y):
        means = []
        stds = []
        priors = []
        
        for i in range(10):
            X_i = X[y == i]
            means.append(np.mean(X_i, axis=0))
            stds.append(np.var(X_i, axis=0, ddof=1) + 10e-3)
            priors.append(len(X_i) / len(X))
        
        self.means = np.array(means)
        self.stds = np.array(stds)
        self.priors = np.array(priors)
        
    def _joint_log_likelihood(self, x):
        n_features = x.shape[0]
        
        log_likelihood = np.zeros(10)
        
        for k in range(10):
            proba = 0
            for i in range(n_features):
                proba += -0.5 * np.log(2 * np.pi * self.stds[k][i])
                proba += -0.5 * ((x[i] - self.means[k][i]) ** 2) / self.stds[k][i]
                
            log_likelihood[k] = proba + np.log(self.priors[k])
        return log_likelihood        
    
    def predict_log_proba(self, X):
        n_samples = X.shape[0]
        
        log_likelihoods = np.zeros((n_samples, 10))
        
        for i in range(n_samples):
            log_likelihoods[i] = self._joint_log_likelihood(X[i])
        
        return log_likelihoods

    def predict_proba(self, X):
        jll = self.predict_log_proba(X)
        log_prob_x = logsumexp(jll, axis=1)
    
        return np.exp(jll - log_prob_x.T)
        
    def predict(self, X):
        return np.argmax(self.predict_log_proba(X), axis=1)
    
    def score(self, X, y):
        return np.mean(self.predict(X) == y)
    


In [133]:
model = GaussianNB()
model.fit(x_train, y_train)

accuracy = model.score(x_test, y_test)
print(f'Accuracy score: {accuracy}')

Accuracy score: 0.7234


In [140]:
np.sum(model.predict_proba(x_test[:1]))

1.0